Load dữ liệu lên bằng thư viện pandas

In [21]:
import pandas as pd
df = pd.read_csv('Badminton.csv')
df.head()

,name,main_category,sub_category,image,link,ratings,no_of_ratings,discount_price,actual_price
0,Jaspo Kidzyy Steel Badminton Racquet with 6pc....,sports & fitness,Badminton,https://m.media-amazon.com/images/I/816z1d69TW...,https://www.amazon.in/Kidzyy-Badminton-Racquet...,3.4,3,₹426,₹800
1,"Klapp Double Shaft Badminton Rackets, with 20 ...",sports & fitness,Badminton,https://m.media-amazon.com/images/I/71UA11w-Ci...,https://www.amazon.in/Klapp-Double-Badminton-R...,3.8,172,₹649,"₹1,899"
2,Asics Gel-Courtmov+ Men's Badminton Shoe - Azu...,sports & fitness,Badminton,https://m.media-amazon.com/images/I/61oMT8Ddsa...,https://www.amazon.in/Asics-Gel-Courtmov-Mens-...,NaN,NaN,"₹3,219","₹3,799"
3,FEROC Fighter Aluminum Badminton Racquet Set o...,sports & fitness,Badminton,https://m.media-amazon.com/images/I/6151mW348F...,https://www.amazon.in/Fighter-Aluminum-Badmint...,3.5,933,₹349,₹699
4,Amazon Brand - Symactive Badminton Kit with 2 ...,sports & fitness,Badminton,https://m.media-amazon.com/images/I/812D6PJq1L...,https://www.amazon.in/Amazon-Brand-Symactive-S...,3.8,36,₹619,"₹1,179"


Lấy các cột cần thiết để đổ về Mysql

In [22]:
data=df.loc[:,['name', 'main_category', 'sub_category','ratings', 'actual_price']]
data.head()


,name,main_category,sub_category,ratings,actual_price
0,Jaspo Kidzyy Steel Badminton Racquet with 6pc....,sports & fitness,Badminton,3.4,₹800
1,"Klapp Double Shaft Badminton Rackets, with 20 ...",sports & fitness,Badminton,3.8,"₹1,899"
2,Asics Gel-Courtmov+ Men's Badminton Shoe - Azu...,sports & fitness,Badminton,NaN,"₹3,799"
3,FEROC Fighter Aluminum Badminton Racquet Set o...,sports & fitness,Badminton,3.5,₹699
4,Amazon Brand - Symactive Badminton Kit with 2 ...,sports & fitness,Badminton,3.8,"₹1,179"


In [23]:
data.dtypes

name             object
main_category    object
sub_category     object
ratings          object
actual_price     object
dtype: object

Kết nối tới mysql thông qua libs mysql.conector

In [28]:
import mysql.connector as msql
# Thay các giá trị này bằng thông tin kết nối của bạn
conn = msql.connect(host='172.16.30.103', port=3306, user='hoang2',
                        password='Hoang@123', database='amazon')
cursor = conn.cursor()
# Tạo bảng để dữ liệu được mapping về bảng sau:
cursor.execute("CREATE TABLE IF NOT EXISTS badminton (id INT AUTO_INCREMENT, name TEXT, main_category TEXT, sub_category TEXT, ratings TEXT,actual_price TEXT, PRIMARY KEY (id));")
print("Table is created or already exists....")

Table is created or already exists....


Insert dữ liệu vào bảng đã được,chia dữ liệu thành các lộ để tránh việc cao tải

In [25]:
# Chuyển dữ liệu từ DataFrame thành danh sách các tuples
data_to_insert = [tuple(x) for x in data.to_numpy()]

# Chia dữ liệu thành các lô (batches) của 200 dòng và chèn vào cơ sở dữ liệu
batch_size = 200
for i in range(0, len(data_to_insert), batch_size):
    batch = data_to_insert[i:i + batch_size]
    sql = "INSERT INTO badminton (name, main_category, sub_category, ratings, actual_price) VALUES (%s, %s, %s, %s, %s) ON DUPLICATE KEY UPDATE name=name"  # Thay đổi phần ON DUPLICATE KEY UPDATE nếu cần
    cursor.executemany(sql, batch)

# Lưu thay đổi và đóng kết nối
conn.commit()
conn.close()